In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch import nn, optim
from torch import nn

In [134]:
# 定义超参数
# 样本量大小
batch_size = 1
# 学习率
learning_rate = 0.05
# 迭代的轮数
num_epoches = 50

In [138]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 二维卷积层，设置输入通道与输出通道，核大小，填充行数与列数
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1,padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=2, stride=1,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1,padding=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2)
        '''self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5)
        '''
        # 采用最大池化
        self.MP = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        # ReLU函数的定义
        self.relu = nn.ReLU()
        # 对全连接层输入与输出的张量的大小的设置
        self.fc1 = nn.Linear(1, 128)
        self.fc2 = nn.Linear(128, 10)
        # 运用softmax函数
        self.logsoftmax = nn.LogSoftmax()
        self.dropout=nn.Dropout(p=0.25)

    # 正向传递
    def forward(self, x):
        in_size = x.size(0)
        # 这里对输入进行逐层处理
        out = self.relu(self.conv1(x))
        print("第一层卷积后的形状:",out.shape)
        out = self.MP(out)
        print("第一层池化后的形状:",out.shape)
        out=self.dropout(out)
        out = self.conv2(out)
        print("第二层卷积后的形状:", out.shape)
        out = self.MP(self.relu(out))
        print("第二层池化后的形状:", out.shape)
        out = self.dropout(out)
        out=self.conv3(out)
        print("第三层卷积后的形状:", out.shape)
        out = self.MP(self.relu(out))
        print("第三层池化后的形状:", out.shape)
        out = self.dropout(out)
        out = self.conv4(out)
        print("第四层卷积后的形状:", out.shape)
        out = self.MP(self.relu(out))
        print("第四层池化后的形状:", out.shape)
        out = self.dropout(out)
        # 返回一个输出的视图，该视图的维度为输入的大小*自动计算的大小
        out = out.view(in_size, -1)
        out = self.relu(out)
        out = self.fc2(out)
        print("最终的输出形状:",out.shape)
        return self.logsoftmax(out)

In [142]:
# 数据预处理。transforms.ToTensor()将图片转换成PyTorch中处理的对象Tensor,并且进行标准化（数据在0~1之间）
# transforms.Normalize()做归一化。它进行了减均值，再除以标准差。两个参数分别是均值和标准差
# transforms.Compose()函数则是将各种预处理的操作组合到了一起
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
# 加载指定的MNIST数据集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=data_tf, download=True)
# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# 选择CNN模型
model = CNN()
# 将交叉熵函数作为损失函数
criterion = nn.CrossEntropyLoss()
# 使用随机梯度下降法
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# 训练模型
epoch = 0
# 利用数据加载器进行迭代
for data in train_loader:
    data[0]=data[0].reshape(1,28, 28)
    img, label = data
    # 将img转化为variable类型，使之可以进行反向传播
    img = Variable(img)
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()
    else:
        img = Variable(img)
        label = Variable(label)
    out = model(img)
    # 计算损失
    loss = criterion(out, label)
    print_loss = loss.data.item()
    # 清除历史梯度，因为梯度在不断累积，而每次迭代不需要继续这种累积
    optimizer.zero_grad()
    # 从损失开始进行反向传播
    loss.backward()
    # 进行一次参数的更新
    optimizer.step()
    # 进而迭代轮数增加1
    epoch += 1
    if epoch % 1 == 0:
        print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
        print(">>>"+"-"*20)
    if epoch==num_epoches:
        break

第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: torch.Size([1, 10])
epoch: 1, loss: 2.507
>>>--------------------
第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: torch.Size([1, 10])
epoch: 2, loss: 2.384
>>>--------------------
第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: t

d:\anaconda\envs\class_face\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: torch.Size([1, 10])
epoch: 37, loss: 1.168
>>>--------------------
第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: torch.Size([1, 10])
epoch: 38, loss: 1.008
>>>--------------------
第一层卷积后的形状: torch.Size([16, 28, 28])
第一层池化后的形状: torch.Size([16, 14, 14])
第二层卷积后的形状: torch.Size([32, 15, 15])
第二层池化后的形状: torch.Size([32, 7, 7])
第三层卷积后的形状: torch.Size([64, 7, 7])
第三层池化后的形状: torch.Size([64, 3, 3])
第四层卷积后的形状: torch.Size([128, 3, 3])
第四层池化后的形状: torch.Size([128, 1, 1])
最终的输出形状: